In [1]:
import pandas as pd
import requests
from io import StringIO

def getCurie_Disease(name):
    itemRequest = 'https://name-resolution-sri.renci.org/lookup?string=' + name + '&autocomplete=false&offset=0&limit=10&biolink_type=DiseaseOrPhenotypicFeature'
    returned = (pd.read_json(StringIO(requests.get(itemRequest).text)))
    resolvedName = returned.curie
    resolvedLabel = returned.label
    return resolvedName, resolvedLabel

def getCurie_Drug(name):
    itemRequest = 'https://name-resolution-sri.renci.org/lookup?string=' + name + '&autocomplete=false&offset=0&limit=10&biolink_type=ChemicalOrDrugOrTreatment'
    returned = (pd.read_json(StringIO(requests.get(itemRequest).text)))
    resolvedName = returned.curie
    resolvedLabel = returned.label
    return resolvedName, resolvedLabel

def build_string_from_list(list):
    outString = "["
    for item in list:
        outString += item + ", "
    outString = outString[:-2] + "]"
    return outString


diseaseData = pd.read_excel('../drug-disease-pairs-pmda.xlsx')

diseaseLabelList = []
diseaseIDList = []
diseaseList = []
drugList = []
drugIDList = []
drugLabelList = []

for index, row in diseaseData.iterrows():
    curr_row_diseasesTreated = row['diseases']
    if type(curr_row_diseasesTreated)!=float:
        print(index)
        curr_row_drugsInTherapy = row['drug active ingredients']
        curr_row_disease_ids = []
        curr_row_disease_id_labels = []
        curr_row_diseaseList = curr_row_diseasesTreated.replace("[","").replace("]","").replace('\'','').split(',')
        #print("disease list: ", curr_row_diseaseList)
        
        try:
            drugCurie,drugLabel = getCurie_Drug(curr_row_drugsInTherapy)
            drugID = drugCurie[0]
            drugIDLabel = drugLabel[0]
                
        except:
            print("could not identify drug: ", curr_row_drugsInTherapy)
            drugID = "NameRes Failed"
            drugIDLabel = "NameRes Failed"

        for idx2,item in enumerate(curr_row_diseaseList):
            item = item.strip().replace(" \n","").replace(" (PREVENTATIVE)","")
            curr_row_diseaseList[idx2] = item
            #print(item)
            
            try:
                print("item being evaluated: ",item)
                diseaseCurie,diseaseLabel = getCurie_Disease(item)
                print(diseaseCurie[0])
                diseaseIDList.append(diseaseCurie[0])
                diseaseLabelList.append(diseaseLabel[0])
                diseaseList.append(item)
                drugList.append(curr_row_drugsInTherapy)
                drugIDList.append(drugID)
                drugLabelList.append(drugIDLabel)
                
            except:
                print("error during name resolving")

sheetData = pd.DataFrame(data=[diseaseIDList, diseaseLabelList, diseaseList, drugList, drugIDList, drugLabelList]).transpose()
sheetData.columns = ['disease IDs', 'disease ID labels', 'list of diseases', 'active ingredients in therapy', 'drug ID', 'drug ID Label']
print(sheetData)
sheetData.to_excel("indication-list-pmda-v1.xlsx")

0
item being evaluated:  ULCERATIVE COLITIS (EXCLUDING SEVERE CASES)
MONDO:0005101
1
item being evaluated:  PAROXYSMAL NOCTURNAL HAEMOGLOBINURIA
MONDO:0100244
2
item being evaluated:  PAROXYSMAL NOCTURNAL HAEMOGLOBINURIA
MONDO:0100244
3
item being evaluated:  CHRONIC GRAFT-VERSUS-HOST DISEASE
MONDO:0020547
4
item being evaluated:  FAMILIAL HYPERCHOLESTEROLEMIA
MONDO:0005439
item being evaluated:  HYPERCHOLESTEROLEMIA
HP:0003124
5
item being evaluated:  HOMOZYGOUS FAMILIAL HYPERCHOLESTEROLEMIA
MONDO:0018328
6
item being evaluated:  CONGENITAL THROMBOTIC THROMBOCYTOPENIC PURPURA
MONDO:0010122
7
item being evaluated:  PREGNANCY
MONDO:0000755
8
item being evaluated:  GENERALIZED MYASTHENIA GRAVIS
MONDO:0009688
9
item being evaluated:  PARTIAL SEIZURES (INCLUDING SECONDARY GENERALIZED SEIZURES)
MONDO:0005754
item being evaluated:  TONIC-CLONIC SEIZURE (ADJUNCTIVE THERAPY)
MONDO:0005754
10
item being evaluated:  GENERALIZED MYASTHENIA GRAVIS
MONDO:0009688
11
item being evaluated:  DISTAL MYO